In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
# import tensorflow as tf
from tensorflow.keras.models import load_model
from skimage import metrics

#### Loading Validation Dataset

In [ ]:
path = os.path.abspath("..")

X_val = np.load(path+"/train_test_data/X_val.npy")
y_val = np.load (path+"/train_test_data/y_val.npy")

## New validation model data from new simulation to validate the model
# X_val = np.load(path+"/quantification/validation_dataset/X.npy")
# y_val = np.load (path+"/quantification/validation_dataset/y.npy")

X_val.shape, y_val.shape

#### Loading Trained Model

In [ ]:
model = load_model(path+"/lstm_model_2.h5")
history = np.load(path+"/history_2.npy", allow_pickle=True).item()

model.summary()

#### Creating function for scaling pixel from 0 to 255 (Needed for Simalirity Index Measurement)

In [ ]:
## Scaling the array values from 0 to 255 for 0 as black pixel and 255 as white pixel for calculation of structural similarity
## Scaling is done to both Ground Truth and Predicted Frame
def scale_array(arr):
    arr_min = np.min(arr)  ## Flattens the array and finds the min value inside it
    arr_max = np.max(arr)  ## Flattens the array and finds the max value inside it
    scaled_array = (arr - arr_min) / (arr_max - arr_min) * 255
    scaled_array = scaled_array.astype(np.uint8)
    
    return scaled_array

# GEN Zero  [3 PF$_{frames}$ --> 1 PD$^{G_0}$$_{frame}$] 
(All 3 PF$_{frames}$ are from Phase Field Simulation)

#### Prediction

In [ ]:
GT_array = y_val
a, b, c, d, e = y_val.shape[0], y_val.shape[1], y_val.shape[2], y_val.shape[3], y_val.shape[4] 
PD_array = np.zeros((a, b, c, d, e))   ## Predicted arrays

for i in range(X_val.shape[0]):
  frames = X_val[i]   ## taking 1 validation video sample at one time
  PD_array[i] = model.predict(np.expand_dims(frames, axis=0), verbose=0, workers=10, use_multiprocessing=True)

GT_array.shape, PD_array.shape

#### Visualization

In [ ]:
vs = 10 # Video frame to look for visualtization

fig, axes = plt.subplots(2, 17, figsize=(28, 4))
fig.suptitle(f"Generation 1 Prediction of Video Sequence No: {vs}" , fontsize=20)
for idx, ax in enumerate(axes[0]):
    ax.imshow(np.squeeze(GT_array[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"GT Frame {idx + 4}", size=10)
    ax.axis("off")

for idx, ax in enumerate(axes[1]):
    ax.imshow(np.squeeze(PD_array[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"Pred Frame {idx + 4}", size=8)
    ax.axis("off")
plt.show()

### Quantification

In [ ]:
## Scaling the array values from 0 to 255 for 0 as black pixel and 255 as white pixel for calculation of structural similarity
## Scaling is done to both Ground Truth and Predicted Frame

GT_scaled = scale_array(y_val)
PD_scaled = scale_array(PD_array)

#### For Individual Image Frames

In [ ]:
## For Individual Image Frames
mse = np.zeros((a,b))
rmse = np.zeros((a,b))
psnr = np.zeros((a,b))
ssim = np.zeros((a,b))

for i in range(GT_scaled.shape[0]):
  for j in range(GT_scaled.shape[1]):
    mse[i][j] = metrics.mean_squared_error(GT_scaled[i,j,:,:,0], PD_scaled[i,j,:,:,0])
    rmse[i][j] = metrics.normalized_root_mse(GT_scaled[i,j,:,:,0], PD_scaled[i,j,:,:,0])
    psnr[i][j] = metrics.peak_signal_noise_ratio(GT_scaled[i,j,:,:,0], PD_scaled[i,j,:,:,0], data_range=None)
    ssim[i][j] = metrics.structural_similarity(GT_scaled[i,j,:,:,0], PD_scaled[i,j,:,:,0], win_size=3)


#### For Individual Video Sequence (17 frames)

In [ ]:
## For Individual Video Sequence (17 frames)
mse_vs = np.zeros((a))
rmse_vs = np.zeros((a))
psnr_vs = np.zeros((a))
ssim_vs = np.zeros((a))

for i in range(GT_scaled.shape[0]):
  # for j in range(GT_scaled.shape[1]):
  mse_vs[i] = metrics.mean_squared_error(GT_scaled[i,:,:,:,0], PD_scaled[i,:,:,:,0])
  rmse_vs[i] = metrics.normalized_root_mse(GT_scaled[i,:,:,:,0], PD_scaled[i,:,:,:,0])
  psnr_vs[i] = metrics.peak_signal_noise_ratio(GT_scaled[i,:,:,:,0], PD_scaled[i,:,:,:,0], data_range=None)
  ssim_vs[i] = metrics.structural_similarity(GT_scaled[i,:,:,:,0], PD_scaled[i,:,:,:,0], win_size=3)


#### For ALL Validation set at once

In [ ]:
## For ALL Validation set at once
mse_all = metrics.mean_squared_error(GT_scaled, PD_scaled)
rmse_all = metrics.normalized_root_mse(GT_scaled, PD_scaled)
psnr_all = metrics.peak_signal_noise_ratio(GT_scaled, PD_scaled, data_range=None)
ssim_all = metrics.structural_similarity(GT_scaled[:,:,:,:,0], PD_scaled[:,:,:,:,0], win_size=3)

print(f'Average Mean Square Error of all the validation set is: {mse_all}')
print(f'Average Root Mean Square Error of all the validation set is: {rmse_all}')
print(f'Average Peak Signal to Noise Ratio of all the validation set is: {psnr_all}')
print(f'Average Structural Simalirity Index of all the validation set is: {ssim_all}')

## Plotting

### For Individual Frames

In [ ]:
plt.rcParams["figure.figsize"] = (6.5,5)

In [ ]:
frame_no = np.arange(mse.shape[1])+1

for i in range(mse.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, mse[i], '-s',)# label=f"VS$_{i}$")

plt.title("MSE of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("MSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(rmse.shape[1])+1

for i in range(rmse.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, rmse[i], '-s',)# label=f"VS$_{i}$")

plt.title("RMSE of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("RMSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(psnr.shape[1])+1

for i in range(psnr.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, psnr[i], '-s',)# label=f"VS$_{i}$")

plt.title("PSNR of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("PSNR", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(ssim.shape[1])+1

for i in range(ssim.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, ssim[i], '-s',)# label=f"VS$_{i}$")

plt.title("SSIM of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


### For Individual Video Sequence

In [ ]:
vs_no = np.arange(mse_vs.shape[0])+1

plt.plot(vs_no, mse_vs, '-o', color='red')
plt.title("MSE of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("MSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(rmse_vs.shape[0])+1

plt.plot(vs_no, rmse_vs, '-o', color='red')
plt.title("RMSE of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("RMSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(psnr_vs.shape[0])+1

plt.plot(vs_no, psnr_vs, '-o', color='red')
plt.title("PSNR of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("PSNR", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(ssim_vs.shape[0])+1

plt.plot(vs_no, ssim_vs, '-o', color='red')
plt.title("SSIM of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

### 2 plots in 1 fig

In [ ]:

vs_no = np.arange(len(ssim_vs))+1

fig, ax1 = plt.subplots()
plt.xticks(fontsize=14, rotation=0)

# Plot the first array on the left y-axis
ax1.plot(vs_no, ssim_vs, '-o', color='tab:blue')
ax1.set_xlabel("Video Sequences", size=20)
ax1.set_ylabel("SSIM", size=20, color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue', )
# ax1.set_ylim([min(ssim_vs)*0.3, 1])  
ax1.set_ylim([0.8, 1])
plt.yticks(fontsize=14, rotation=0)

ax1.xaxis.set_major_locator(plt.MaxNLocator(6,))
ax1.yaxis.set_major_locator(plt.MaxNLocator(5))


# Create a second y-axis on the right side
ax2 = ax1.twinx()

# Plot the second array on the right y-axis
ax2.plot(vs_no, rmse_vs, '-s', color='tab:red')
ax2.set_ylabel("RMSE", size=20, color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
# ax2.set_ylim([min(rmse_vs)*0.99, max(rmse_vs)*1.05])  
ax2.set_ylim([0.05, 0.1])
plt.yticks(fontsize=14, rotation=0)

ax2.yaxis.set_major_locator(plt.MaxNLocator(5))

# Add labels to the plot
# plt.title('SSIM & RMSE In all Video Seuences',size=20)

plt.tight_layout()
plt.savefig('figures/ssim_vs_rmse_G0.png')
plt.show()


In [ ]:
vs_no = np.arange(len(mse_vs))

fig, ax1 = plt.subplots()
plt.xticks(fontsize=14, rotation=0)

# Plot the first array on the left y-axis
ax1.plot(vs_no, mse_vs, '-o', color='tab:blue')
ax1.set_xlabel("Video Sequences", size=20)
ax1.set_ylabel("MSE", size=20, color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue', )
# ax1.set_ylim([min(mse_vs)*0.4, max(mse_vs)*1.05])  
ax1.set_ylim([0, 120])
plt.yticks(fontsize=14, rotation=0)

ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
ax1.yaxis.set_major_locator(plt.MaxNLocator(6))

# Create a second y-axis on the right side
ax2 = ax1.twinx()

# Plot the second array on the right y-axis
ax2.plot(vs_no, psnr_vs, '-s', color='tab:red')
ax2.set_ylabel("PSNR", size=20, color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
# ax2.set_ylim([min(psnr_vs)*0.95, max(psnr_vs)*1.2])  
ax2.set_ylim([27, 45])
plt.yticks(fontsize=14, rotation=0)

ax2.yaxis.set_major_locator(plt.MaxNLocator(5))

# Add labels to the plot
# plt.title('MSE & PSNR In all Video Seuences',size=20)
# plt.tight_layout()

plt.show()


# Generation 1

## Method 0 
## For Generation 1 prediction We also inherit Generation 0's predicted result as parent. 
## GEN One  [(2 PF + 1 PD$^{G_0}$)$_{frame}$  --> 1 PD$^{G_1}$$_{frame}$] 
(2 PF$_{frames}$ are from Phase Field Simulation & 1 PD$^{G_0}$$_{frame}$ is from Generation 0 predicted frame)

In [ ]:
## Generation 1 of prediction
## The new frame from prediction is added to the ending frame of X_val and first frame of X_val is removed. 
## i.e. X_val had f1, f2, f3 and predicted fp4. 
## Gen 1 input has f2, f3, fp4 and predicvts fp5

X_g1 = np.delete(np.concatenate((X_val, PD_array), axis=-1), 0, axis=-1)
X_g1.shape

In [ ]:
## We remove the last frame of every video sequence as now the 3 frames to predict next frame consists of gen0 prediction 
## i.e. we are using f18, f19, fp20 to predict fp21, but the problem here is we don't gave GT for fp21 
## So, we remove the last frame of every video sequence hence we limit our predictions for 16 frames making the last input as f17, f18, fp19 to predict fp20

X_g1 = np.delete(X_g1, -1, axis=1)
X_g1.shape

In [ ]:
## We similarly remove the 1st frame of GT video sequence i.e. frame 4 of every video sequence
## Because, we are using f2, f2, fp4 to predict fp5. So we need f5 of GT to quantify with hence we can remove f4 of GT has it was quantified in Generation 0
y_g1 = np.delete(y_val, 0, axis=1)
y_g1.shape

In [ ]:
a, b, c, d, e = y_g1.shape[0], y_g1.shape[1], y_g1.shape[2], y_g1.shape[3], y_g1.shape[4] 
PD_G1 = np.zeros((a, b, c, d, e))   ## Predicted arrays

for i in range(X_g1.shape[0]):
  frames = X_g1[i]   ## taking 1 validation video sample at one time
  PD_G1[i] = model.predict(np.expand_dims(frames, axis=0), verbose=0, workers=10, use_multiprocessing=True)

y_g1.shape, PD_G1.shape

In [ ]:
## Scaling the array values from 0 to 255 for 0 as black pixel and 255 as white pixel for calculation of structural similarity
## Scaling is done to both Ground Truth and Predicted Frame
def scale_array(arr):
    arr_min = np.min(arr)  ## Flattens the array and finds the min value inside it
    arr_max = np.max(arr)  ## Flattens the array and finds the max value inside it
    scaled_array = (arr - arr_min) / (arr_max - arr_min) * 255
    scaled_array = scaled_array.astype(np.uint8)
    
    return scaled_array

y_g1_scaled = scale_array(y_g1)
PD_G1_scaled = scale_array(PD_G1)

In [ ]:
vs = 30 # Video frame to look for visualtization

fig, axes = plt.subplots(2, 16, figsize=(25, 4))
fig.suptitle(f"Generation 1 Prediction of Video Sequence No: {vs}" , fontsize=20)
for idx, ax in enumerate(axes[0]):
    ax.imshow(np.squeeze(y_g1[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"GT Frame {idx + 5}", size=10)
    ax.axis("off")

for idx, ax in enumerate(axes[1]):
    ax.imshow(np.squeeze(PD_G1[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"Pred Frame {idx + 5}", size=8)
    ax.axis("off")
plt.show()

### Quantification

In [ ]:
## For Individual Image Frames
mse_g1 = np.zeros((a,b))
rmse_g1 = np.zeros((a,b))
psnr_g1 = np.zeros((a,b))
ssim_g1 = np.zeros((a,b))

for i in range(y_g1_scaled.shape[0]):
  for j in range(y_g1_scaled.shape[1]):
    mse_g1[i][j] = metrics.mean_squared_error(y_g1_scaled[i,j,:,:,0], PD_G1_scaled[i,j,:,:,0])
    rmse_g1[i][j] = metrics.normalized_root_mse(y_g1_scaled[i,j,:,:,0], PD_G1_scaled[i,j,:,:,0])
    psnr_g1[i][j] = metrics.peak_signal_noise_ratio(y_g1_scaled[i,j,:,:,0], PD_G1_scaled[i,j,:,:,0], data_range=None)
    ssim_g1[i][j] = metrics.structural_similarity(y_g1_scaled[i,j,:,:,0], PD_G1_scaled[i,j,:,:,0], win_size=3)


In [ ]:
## For Individual Video Sequence (17 frames)
mse_g1_vs = np.zeros((a))
rmse_g1_vs = np.zeros((a))
psnr_g1_vs = np.zeros((a))
ssim_g1_vs = np.zeros((a))

for i in range(y_g1_scaled.shape[0]):
  # for j in range(y_g1_scaled.shape[1]):
  mse_g1_vs[i] = metrics.mean_squared_error(y_g1_scaled[i,:,:,:,0], PD_G1_scaled[i,:,:,:,0])
  rmse_g1_vs[i] = metrics.normalized_root_mse(y_g1_scaled[i,:,:,:,0], PD_G1_scaled[i,:,:,:,0])
  psnr_g1_vs[i] = metrics.peak_signal_noise_ratio(y_g1_scaled[i,:,:,:,0], PD_G1_scaled[i,:,:,:,0], data_range=None)
  ssim_g1_vs[i] = metrics.structural_similarity(y_g1_scaled[i,:,:,:,0], PD_G1_scaled[i,:,:,:,0], win_size=3)


In [ ]:
## For ALL Validation set at once
mse_g1_all = metrics.mean_squared_error(y_g1_scaled, PD_G1_scaled)
rmse_g1_all = metrics.normalized_root_mse(y_g1_scaled, PD_G1_scaled)
psnr_g1_all = metrics.peak_signal_noise_ratio(y_g1_scaled, PD_G1_scaled, data_range=None)
ssim_g1_all = metrics.structural_similarity(y_g1_scaled[:,:,:,:,0], PD_G1_scaled[:,:,:,:,0], win_size=3)

print(f'Average Mean Square Error of all the validation set is: {mse_g1_all}')
print(f'Average Root Mean Square Error of all the validation set is: {rmse_g1_all}')
print(f'Average Peak Signal to Noise Ratio of all the validation set is: {psnr_g1_all}')
print(f'Average Structural Simalirity Index of all the validation set is: {ssim_g1_all}')

## Plotting

### For Individual Frames

In [ ]:
frame_no = np.arange(mse_g1.shape[1])+1

for i in range(mse_g1.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, mse_g1[i], '-s',)# label=f"VS$_{i}$")

plt.title("MSE of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("MSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(rmse_g1.shape[1])+1

for i in range(rmse_g1.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, rmse_g1[i], '-s',)# label=f"VS$_{i}$")

plt.title("RMSE of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("RMSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(psnr_g1.shape[1])+1

for i in range(psnr_g1.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, psnr_g1[i], '-s',)# label=f"VS$_{i}$")

plt.title("PSNR of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("PSNR", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(ssim_g1.shape[1])+1

for i in range(ssim_g1.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, ssim_g1[i], '-s',)# label=f"VS$_{i}$")

plt.title("SSIM of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


### For Individual Video Sequence

In [ ]:
vs_no = np.arange(mse_g1_vs.shape[0])+1

plt.plot(vs_no, mse_g1_vs, '-o', color='red')
plt.title("MSE of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("MSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(rmse_g1_vs.shape[0])+1

plt.plot(vs_no, rmse_g1_vs, '-o', color='red')
plt.title("RMSE of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("RMSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(psnr_g1_vs.shape[0])+1

plt.plot(vs_no, psnr_g1_vs, '-o', color='red')
plt.title("PSNR of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("PSNR", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(ssim_g1_vs.shape[0])+1

plt.plot(vs_no, ssim_g1_vs, '-o', color='red')
plt.title("SSIM of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

### 2 plots in 1 fig

In [ ]:
vs_no = np.arange(len(ssim_g1_vs))+1

fig, ax1 = plt.subplots()
plt.xticks(fontsize=14, rotation=0)

# Plot the first array on the left y-axis
ax1.plot(vs_no, ssim_g1_vs, '-o', color='tab:blue')
ax1.set_xlabel("Video Sequence No", size=20)
ax1.set_ylabel("SSIM", size=20, color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue', )
# ax1.set_ylim([min(ssim_g1_vs)*0.3, 1])  
ax1.set_ylim([0.75, 1])
plt.yticks(fontsize=14, rotation=0)

ax1.xaxis.set_major_locator(plt.MaxNLocator(6,))
ax1.yaxis.set_major_locator(plt.MaxNLocator(5))


# Create a second y-axis on the right side
ax2 = ax1.twinx()

# Plot the second array on the right y-axis
ax2.plot(vs_no, rmse_g1_vs, '-s', color='tab:red')
ax2.set_ylabel("RMSE", size=20, color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
# ax2.set_ylim([min(rmse_g1_vs)*0.99, max(rmse_g1_vs)*1.05]) 
ax2.set_ylim([0.07, 0.17])
plt.yticks(fontsize=14, rotation=0)

# Add labels to the plot
# plt.title('SSIM & RMSE In all Video Seuences',size=20)

plt.tight_layout()
# plt.savefig('ssim_vs_rmse.png')
plt.show()


In [ ]:
vs_no = np.arange(len(mse_g1_vs))

fig, ax1 = plt.subplots()
plt.xticks(fontsize=14, rotation=0)

# Plot the first array on the left y-axis
ax1.plot(vs_no, mse_g1_vs, '-o', color='tab:blue')
ax1.set_xlabel("Video Sequence No", size=20)
ax1.set_ylabel("MSE", size=20, color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue', )
# ax1.set_ylim([min(mse_g1_vs)*0.8, max(mse_g1_vs)*1.02]) 
ax1.set_ylim([0, 270])
plt.yticks(fontsize=14, rotation=0)

ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
ax1.yaxis.set_major_locator(plt.MaxNLocator(6))

# Create a second y-axis on the right side
ax2 = ax1.twinx()

# Plot the second array on the right y-axis
ax2.plot(vs_no, psnr_g1_vs, '-s', color='tab:red')
ax2.set_ylabel("PSNR", size=20, color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
# ax2.set_ylim([min(psnr_g1_vs)*0.99, max(psnr_g1_vs)*1.05])  
ax2.set_ylim([23, 35])
plt.yticks(fontsize=14, rotation=0)

ax2.yaxis.set_major_locator(plt.MaxNLocator(5))

# Add labels to the plot
# plt.title('MSE & PSNR In all Video Seuences',size=20)
# plt.tight_layout()

plt.show()


## Method 1

In [ ]:
# X_val.shape

In [ ]:
# A = X_val[0:1,:,:,:,:]
# A.shape

In [ ]:
# B = model.predict(A, verbose=0, workers=10, use_multiprocessing=True)
# B.shape

In [ ]:
# A = X_val[0:1,:,:,:,:]
# C = np.zeros((1, 1, 64, 64, 1))

# for i in range(X_val.shape[1]):
#   B = model.predict(A, verbose=0, workers=10, use_multiprocessing=True)
#   A = np.delete(np.concatenate((A,B), axis=-1), 0, axis=-1)
#   C = np.concatenate((C,B), axis=1)
  
# C = np.delete(C,0,axis=1)

In [ ]:
# fig, axes = plt.subplots(2, 5, figsize=(20, 8))

# fig.suptitle(f"Generational Prediction " , fontsize=14)

# for idx, ax in enumerate(axes[0]):
#     ax.imshow(np.squeeze(y_val[0:1,idx:idx+1,:,:,:]), cmap="gray")
#     ax.set_title(f"Frame {idx + 3}")
#     ax.axis("off")

# for idx, ax in enumerate(axes[1]):
#     ax.imshow(np.squeeze(C[0:1,idx:idx+1,:,:,:]), cmap="gray")
#     ax.set_title(f"Frame {idx + 3}")
#     ax.axis("off")

# plt.show()

# Generation 2

## For Generation 2 prediction We also inherit Generation 0's and Generation 1's predicted result as parent. 
## GEN 2 ==> [(1 PF + 1 PD$^{G_0}$ + 1 PD$^{G_1}$)$_{frame}$  --> 1 PD$^{G_2}$$_{frame}$] 
(1 PF$_{frame}$ is from Phase Field Simulation & 1 PD$^{G_0}$$_{frame}$ is from Generation 0 predicted frame and 1 PD$^{G_1}$$_{frame}$ is from Generation 1 predicted frame)

In [ ]:
## Generation 2 of prediction
## The new frame from prediction is added to the ending frame of X_g1 and first frame of X_g1 is removed. 
## i.e. X_g1 had f2, f3, fp4 and predicted fp5. 
## Gen 2 input has f3, fp4, fp5 and predicvts fp6

X_g2 = np.delete(np.concatenate((X_g1, PD_G1), axis=-1), 0, axis=-1)
X_g2.shape

In [ ]:
## We remove the last frame of every video sequence as now the 3 frames to predict next frame consists of gen0 prediction 
## i.e. we are using f18, f19, fp20 to predict fp21, but the problem here is we don't gave GT for fp21 
## So, we remove the last frame of every video sequence hence we limit our predictions for 16 frames making the last input as f17, f18, fp19 to predict fp20

X_g2 = np.delete(X_g2, -1, axis=1)
X_g2.shape

In [ ]:
## We similarly remove the 1st frame of GT video sequence i.e. frame 4 of every video sequence
## Because, we are using f2, f2, fp4 to predict fp5. So we need f5 of GT to quantify with hence we can remove f4 of GT has it was quantified in Generation 0
y_g2 = np.delete(y_g1, 0, axis=1)
y_g2.shape

In [ ]:
a, b, c, d, e = y_g2.shape[0], y_g2.shape[1], y_g2.shape[2], y_g2.shape[3], y_g2.shape[4] 
PD_G2 = np.zeros((a, b, c, d, e))   ## Predicted arrays

for i in range(X_g1.shape[0]):
  frames = X_g2[i]   ## taking 1 validation video sample at one time
  PD_G2[i] = model.predict(np.expand_dims(frames, axis=0), verbose=0, workers=10, use_multiprocessing=True)

y_g2.shape, PD_G2.shape

In [ ]:
## Scaling the array values from 0 to 255 for 0 as black pixel and 255 as white pixel for calculation of structural similarity
## Scaling is done to both Ground Truth and Predicted Frame
def scale_array(arr):
    arr_min = np.min(arr)  ## Flattens the array and finds the min value inside it
    arr_max = np.max(arr)  ## Flattens the array and finds the max value inside it
    scaled_array = (arr - arr_min) / (arr_max - arr_min) * 255
    scaled_array = scaled_array.astype(np.uint8)
    
    return scaled_array

y_g2_scaled = scale_array(y_g2)
PD_G2_scaled = scale_array(PD_G2)

In [ ]:
vs = 22 # Video frame to look for visualtization

fig, axes = plt.subplots(2, 15, figsize=(25, 4))
fig.suptitle(f"Generation 1 Prediction of Video Sequence No: {vs}" , fontsize=20)
for idx, ax in enumerate(axes[0]):
    ax.imshow(np.squeeze(y_g2[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"GT Frame {idx + 6}", size=10)
    ax.axis("off")

for idx, ax in enumerate(axes[1]):
    ax.imshow(np.squeeze(PD_G2[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"Pred Frame {idx + 6}", size=8)
    ax.axis("off")
plt.show()

### Quantification

In [ ]:
## For Individual Image Frames
mse_g2 = np.zeros((a,b))
rmse_g2 = np.zeros((a,b))
psnr_g2 = np.zeros((a,b))
ssim_g2 = np.zeros((a,b))

for i in range(y_g2_scaled.shape[0]):
  for j in range(y_g2_scaled.shape[1]):
    mse_g2[i][j] = metrics.mean_squared_error(y_g2_scaled[i,j,:,:,0], PD_G2_scaled[i,j,:,:,0])
    rmse_g2[i][j] = metrics.normalized_root_mse(y_g2_scaled[i,j,:,:,0], PD_G2_scaled[i,j,:,:,0])
    psnr_g2[i][j] = metrics.peak_signal_noise_ratio(y_g2_scaled[i,j,:,:,0], PD_G2_scaled[i,j,:,:,0], data_range=None)
    ssim_g2[i][j] = metrics.structural_similarity(y_g2_scaled[i,j,:,:,0], PD_G2_scaled[i,j,:,:,0], win_size=3)


In [ ]:
## For Individual Video Sequence (17 frames)
mse_g2_vs = np.zeros((a))
rmse_g2_vs = np.zeros((a))
psnr_g2_vs = np.zeros((a))
ssim_g2_vs = np.zeros((a))

for i in range(y_g2_scaled.shape[0]):
  # for j in range(y_g2_scaled.shape[1]):
  mse_g2_vs[i] = metrics.mean_squared_error(y_g2_scaled[i,:,:,:,0], PD_G2_scaled[i,:,:,:,0])
  rmse_g2_vs[i] = metrics.normalized_root_mse(y_g2_scaled[i,:,:,:,0], PD_G2_scaled[i,:,:,:,0])
  psnr_g2_vs[i] = metrics.peak_signal_noise_ratio(y_g2_scaled[i,:,:,:,0], PD_G2_scaled[i,:,:,:,0], data_range=None)
  ssim_g2_vs[i] = metrics.structural_similarity(y_g2_scaled[i,:,:,:,0], PD_G2_scaled[i,:,:,:,0], win_size=3)


In [ ]:
## For ALL Validation set at once
mse_g2_all = metrics.mean_squared_error(y_g2_scaled, PD_G2_scaled)
rmse_g2_all = metrics.normalized_root_mse(y_g2_scaled, PD_G2_scaled)
psnr_g2_all = metrics.peak_signal_noise_ratio(y_g2_scaled, PD_G2_scaled, data_range=None)
ssim_g2_all = metrics.structural_similarity(y_g2_scaled[:,:,:,:,0], PD_G2_scaled[:,:,:,:,0], win_size=3)

print(f'Average Mean Square Error of all the validation set is: {mse_g2_all}')
print(f'Average Root Mean Square Error of all the validation set is: {rmse_g2_all}')
print(f'Average Peak Signal to Noise Ratio of all the validation set is: {psnr_g2_all}')
print(f'Average Structural Simalirity Index of all the validation set is: {ssim_g2_all}')

## Plotting

### For Individual Frames

In [ ]:
frame_no = np.arange(mse_g2.shape[1])+1

for i in range(mse_g2.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, mse_g2[i], '-s',)# label=f"VS$_{i}$")

plt.title("MSE of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("MSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(rmse_g2.shape[1])+1

for i in range(rmse_g2.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, rmse_g2[i], '-s',)# label=f"VS$_{i}$")

plt.title("RMSE of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("RMSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(psnr_g2.shape[1])+1

for i in range(psnr_g2.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, psnr_g2[i], '-s',)# label=f"VS$_{i}$")

plt.title("PSNR of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("PSNR", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(ssim_g2.shape[1])+1

for i in range(ssim_g2.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, ssim_g2[i], '-s',)# label=f"VS$_{i}$")

plt.title("SSIM of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


### For Individual Video Sequence

In [ ]:
vs_no = np.arange(mse_g2_vs.shape[0])+1

plt.plot(vs_no, mse_g2_vs, '-o', color='red')
plt.title("MSE of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("MSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(rmse_g2_vs.shape[0])+1

plt.plot(vs_no, rmse_g2_vs, '-o', color='red')
plt.title("RMSE of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("RMSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(psnr_g2_vs.shape[0])+1

plt.plot(vs_no, psnr_g2_vs, '-o', color='red')
plt.title("PSNR of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("PSNR", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(ssim_g2_vs.shape[0])+1

plt.plot(vs_no, ssim_g2_vs, '-o', color='red')
plt.title("SSIM of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

### 2 plots in 1 fig

In [ ]:
vs_no = np.arange(len(ssim_g2_vs))+1

fig, ax1 = plt.subplots()
plt.xticks(fontsize=14, rotation=0)

# Plot the first array on the left y-axis
ax1.plot(vs_no, ssim_g2_vs, '-o', color='tab:blue')
ax1.set_xlabel("Video Sequence No", size=20)
ax1.set_ylabel("SSIM", size=20, color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue', )
# ax1.set_ylim([min(ssim_g2_vs)*0.2, 1])  
ax1.set_ylim([0.7, 1])
plt.yticks(fontsize=14, rotation=0)

ax1.xaxis.set_major_locator(plt.MaxNLocator(6,))
ax1.yaxis.set_major_locator(plt.MaxNLocator(5))


# Create a second y-axis on the right side
ax2 = ax1.twinx()

# Plot the second array on the right y-axis
ax2.plot(vs_no, rmse_g2_vs, '-s', color='tab:red')
ax2.set_ylabel("RMSE", size=20, color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
# ax2.set_ylim([min(rmse_g2_vs)*0.95, max(rmse_g2_vs)*1.2]) 
ax2.set_ylim([0.088, 0.2])
plt.yticks(fontsize=14, rotation=0)

# Add labels to the plot
# plt.title('SSIM & RMSE In all Video Seuences',size=20)

plt.tight_layout()
# plt.savefig('ssim_vs_rmse.png')
plt.show()


In [ ]:
vs_no = np.arange(len(mse_g2_vs))

fig, ax1 = plt.subplots()
plt.xticks(fontsize=14, rotation=0)

# Plot the first array on the left y-axis
ax1.plot(vs_no, mse_g2_vs, '-o', color='tab:blue')
ax1.set_xlabel("Video Sequence No", size=20)
ax1.set_ylabel("MSE", size=20, color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue', )
# ax1.set_ylim([min(mse_g2_vs)*0.95, max(mse_g2_vs)*1.01])  
ax1.set_ylim([0, 400])
plt.yticks(fontsize=14, rotation=0)

ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
ax1.yaxis.set_major_locator(plt.MaxNLocator(6))

# Create a second y-axis on the right side
ax2 = ax1.twinx()

# Plot the second array on the right y-axis
ax2.plot(vs_no, psnr_g2_vs, '-s', color='tab:red')
ax2.set_ylabel("PSNR", size=20, color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
# ax2.set_ylim([min(psnr_g2_vs)*0.995, max(psnr_g2_vs)*1.01])  
ax2.set_ylim([21.5, 35])
plt.yticks(fontsize=14, rotation=0)

ax2.yaxis.set_major_locator(plt.MaxNLocator(5))

# Add labels to the plot
# plt.title('MSE & PSNR In all Video Seuences',size=20)
# plt.tight_layout()

plt.show()


# Generation 3

## For Generation 3 prediction We inherit Generation 0's, Generation 1's and , Generation 2's predicted result as parent. 
## GEN 3 ==> [(1 PD$^{G_0}$ + 1 PD$^{G_1}$ + 1 PD$^{G_2}$)$_{frame}$  --> 1 PD$^{G_3}$$_{frame}$] 
(1 PD$^{G_0}$$_{frame}$ is from Generation 0 predicted frame, 1 PD$^{G_1}$$_{frame}$ is from Generation 1 predicted frame and 1 PD$^{G_2}$$_{frame}$ is from Generation 2 predicted frame)

At this point we are only inheriting Model Predicted frames as input. So no phase filed generated frame is being used at this point. 

In [ ]:
## Generation 2 of prediction
## The new frame from prediction is added to the ending frame of X_g2 and first frame of X_g2 is removed. 
## i.e. X_g2 had f3, fp4, fp5 and predicted fp6. 
## Gen 2 input has fp4, fp5, fp6 and predicvts fp7

X_g3 = np.delete(np.concatenate((X_g2, PD_G2), axis=-1), 0, axis=-1)
X_g3.shape

In [ ]:
## We remove the last frame of every video sequence as now the 3 frames to predict next frame consists of gen0 prediction 
## i.e. we are using f18, f19, fp20 to predict fp21, but the problem here is we don't gave GT for fp21 
## So, we remove the last frame of every video sequence hence we limit our predictions for 16 frames making the last input as f17, f18, fp19 to predict fp20

X_g3 = np.delete(X_g3, -1, axis=1)
X_g3.shape

In [ ]:
## We similarly remove the 1st frame of GT video sequence i.e. frame 4 of every video sequence
## Because, we are using f2, f2, fp4 to predict fp5. So we need f5 of GT to quantify with hence we can remove f4 of GT has it was quantified in Generation 0
y_g3 = np.delete(y_g2, 0, axis=1)
y_g3.shape

In [ ]:
a, b, c, d, e = y_g3.shape[0], y_g3.shape[1], y_g3.shape[2], y_g3.shape[3], y_g3.shape[4] 
PD_G3 = np.zeros((a, b, c, d, e))   ## Predicted arrays

for i in range(X_g2.shape[0]):
  frames = X_g3[i]   ## taking 1 validation video sample at one time
  PD_G3[i] = model.predict(np.expand_dims(frames, axis=0), verbose=0, workers=10, use_multiprocessing=True)

y_g3.shape, PD_G3.shape

In [ ]:
## Scaling the array values from 0 to 255 for 0 as black pixel and 255 as white pixel for calculation of structural similarity
## Scaling is done to both Ground Truth and Predicted Frame
def scale_array(arr):
    arr_min = np.min(arr)  ## Flattens the array and finds the min value inside it
    arr_max = np.max(arr)  ## Flattens the array and finds the max value inside it
    scaled_array = (arr - arr_min) / (arr_max - arr_min) * 255
    scaled_array = scaled_array.astype(np.uint8)
    
    return scaled_array

y_g3_scaled = scale_array(y_g3)
PD_G3_scaled = scale_array(PD_G3)

In [ ]:
vs = 96 # Video frame to look for visualtization

fig, axes = plt.subplots(2, 14, figsize=(25, 4))
fig.suptitle(f"Generation 1 Prediction of Video Sequence No: {vs}" , fontsize=20)
for idx, ax in enumerate(axes[0]):
    ax.imshow(np.squeeze(y_g3[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"GT Frame {idx + 7}", size=10)
    ax.axis("off")

for idx, ax in enumerate(axes[1]):
    ax.imshow(np.squeeze(PD_G3[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"Pred Frame {idx + 7}", size=8)
    ax.axis("off")
plt.show()

### Quantification

In [ ]:
## For Individual Image Frames
mse_g3 = np.zeros((a,b))
rmse_g3 = np.zeros((a,b))
psnr_g3 = np.zeros((a,b))
ssim_g3 = np.zeros((a,b))

for i in range(y_g3_scaled.shape[0]):
  for j in range(y_g3_scaled.shape[1]):
    mse_g3[i][j] = metrics.mean_squared_error(y_g3_scaled[i,j,:,:,0], PD_G3_scaled[i,j,:,:,0])
    rmse_g3[i][j] = metrics.normalized_root_mse(y_g3_scaled[i,j,:,:,0], PD_G3_scaled[i,j,:,:,0])
    psnr_g3[i][j] = metrics.peak_signal_noise_ratio(y_g3_scaled[i,j,:,:,0], PD_G3_scaled[i,j,:,:,0], data_range=None)
    ssim_g3[i][j] = metrics.structural_similarity(y_g3_scaled[i,j,:,:,0], PD_G3_scaled[i,j,:,:,0], win_size=3)


In [ ]:
## For Individual Video Sequence (17 frames)
mse_g3_vs = np.zeros((a))
rmse_g3_vs = np.zeros((a))
psnr_g3_vs = np.zeros((a))
ssim_g3_vs = np.zeros((a))

for i in range(y_g3_scaled.shape[0]):
  # for j in range(y_g3_scaled.shape[1]):
  mse_g3_vs[i] = metrics.mean_squared_error(y_g3_scaled[i,:,:,:,0], PD_G3_scaled[i,:,:,:,0])
  rmse_g3_vs[i] = metrics.normalized_root_mse(y_g3_scaled[i,:,:,:,0], PD_G3_scaled[i,:,:,:,0])
  psnr_g3_vs[i] = metrics.peak_signal_noise_ratio(y_g3_scaled[i,:,:,:,0], PD_G3_scaled[i,:,:,:,0], data_range=None)
  ssim_g3_vs[i] = metrics.structural_similarity(y_g3_scaled[i,:,:,:,0], PD_G3_scaled[i,:,:,:,0], win_size=3)


In [ ]:
## For ALL Validation set at once
mse_g3_all = metrics.mean_squared_error(y_g3_scaled, PD_G3_scaled)
rmse_g3_all = metrics.normalized_root_mse(y_g3_scaled, PD_G3_scaled)
psnr_g3_all = metrics.peak_signal_noise_ratio(y_g3_scaled, PD_G3_scaled, data_range=None)
ssim_g3_all = metrics.structural_similarity(y_g3_scaled[:,:,:,:,0], PD_G3_scaled[:,:,:,:,0], win_size=3)

print(f'Average Mean Square Error of all the validation set is: {mse_g3_all}')
print(f'Average Root Mean Square Error of all the validation set is: {rmse_g3_all}')
print(f'Average Peak Signal to Noise Ratio of all the validation set is: {psnr_g3_all}')
print(f'Average Structural Simalirity Index of all the validation set is: {ssim_g3_all}')

## Plotting

### For Individual Frames

In [ ]:
frame_no = np.arange(mse_g3.shape[1])+1

for i in range(mse_g3.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, mse_g3[i], '-s',)# label=f"VS$_{i}$")

plt.title("MSE of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("MSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(rmse_g3.shape[1])+1

for i in range(rmse_g3.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, rmse_g3[i], '-s',)# label=f"VS$_{i}$")

plt.title("RMSE of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("RMSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(psnr_g3.shape[1])+1

for i in range(psnr_g3.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, psnr_g3[i], '-s',)# label=f"VS$_{i}$")

plt.title("PSNR of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("PSNR", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


In [ ]:
frame_no = np.arange(ssim_g3.shape[1])+1

for i in range(ssim_g3.shape[0]):   ## We are iteraing over all 137 video samples
    plt.plot(frame_no, ssim_g3[i], '-s',)# label=f"VS$_{i}$")

plt.title("SSIM of video samples", size=20)
plt.xlabel("Frame No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()


### For Individual Video Sequence

In [ ]:
vs_no = np.arange(mse_g3_vs.shape[0])+1

plt.plot(vs_no, mse_g3_vs, '-o', color='red')
plt.title("MSE of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("MSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(rmse_g3_vs.shape[0])+1

plt.plot(vs_no, rmse_g3_vs, '-o', color='red')
plt.title("RMSE of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("RMSE", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(psnr_g3_vs.shape[0])+1

plt.plot(vs_no, psnr_g3_vs, '-o', color='red')
plt.title("PSNR of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("PSNR", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

In [ ]:
vs_no = np.arange(ssim_g3_vs.shape[0])+1

plt.plot(vs_no, ssim_g3_vs, '-o', color='red')
plt.title("SSIM of video samples", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
# plt.legend(loc=(1.01,0.3))

plt.show()

### 2 plots in 1 fig

In [ ]:
vs_no = np.arange(len(ssim_g3_vs))+1

fig, ax1 = plt.subplots()
plt.xticks(fontsize=14, rotation=0)

# Plot the first array on the left y-axis
ax1.plot(vs_no, ssim_g3_vs, '-o', color='tab:blue')
ax1.set_xlabel("Video Sequence No", size=20)
ax1.set_ylabel("SSIM", size=20, color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue', )
# ax1.set_ylim([min(ssim_g3_vs)*0.05, 1]) 
ax1.set_ylim([0.55, 1])
plt.yticks(fontsize=14, rotation=0)

ax1.xaxis.set_major_locator(plt.MaxNLocator(6,))
ax1.yaxis.set_major_locator(plt.MaxNLocator(5))


# Create a second y-axis on the right side
ax2 = ax1.twinx()

# Plot the second array on the right y-axis
ax2.plot(vs_no, rmse_g3_vs, '-s', color='tab:red')
ax2.set_ylabel("RMSE", size=20, color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
# ax2.set_ylim([min(rmse_g3_vs)*0.95, max(rmse_g3_vs)*1.25]) 
ax2.set_ylim([0.11, 0.26])
plt.yticks(fontsize=14, rotation=0)

# Add labels to the plot
# plt.title('SSIM & RMSE In all Video Seuences',size=20)

plt.tight_layout()
# plt.savefig('figures/ssim_vs_rmse_gen3.png')
plt.show()


In [ ]:
vs_no = np.arange(len(mse_g3_vs))

fig, ax1 = plt.subplots()
plt.xticks(fontsize=14, rotation=0)

# Plot the first array on the left y-axis
ax1.plot(vs_no, mse_g3_vs, '-o', color='tab:blue')
ax1.set_xlabel("Video Sequence No", size=20)
ax1.set_ylabel("MSE", size=20, color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue', )
# ax1.set_ylim([min(mse_g3_vs)*0.75, max(mse_g3_vs)*1.01])  
ax1.set_ylim([0, 680])
plt.yticks(fontsize=14, rotation=0)

ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
ax1.yaxis.set_major_locator(plt.MaxNLocator(6))

# Create a second y-axis on the right side
ax2 = ax1.twinx()

# Plot the second array on the right y-axis
ax2.plot(vs_no, psnr_g3_vs, '-s', color='tab:red')
ax2.set_ylabel("PSNR", size=20, color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
# ax2.set_ylim([min(psnr_g3_vs)*0.995, max(psnr_g3_vs)*1.1]) 
ax2.set_ylim([19.5, 35])
plt.yticks(fontsize=14, rotation=0)

ax2.yaxis.set_major_locator(plt.MaxNLocator(5))

# Add labels to the plot
# plt.title('MSE & PSNR In all Video Seuences',size=20)
# plt.tight_layout()

plt.show()


# Generational Comparision

## Gen 0 Vs Gen 1

In [ ]:
frame_no = np.arange(ssim_vs.shape[0])

plt.rcParams["figure.figsize"] = (20,5)

plt.plot(frame_no, ssim_vs, '-o', color='red', label='Generation 0')
plt.plot(frame_no, ssim_g1_vs,'--s', color='green', label='Generation 1')

plt.title("SSIM: 'Gen 0' Vs 'Gen 1'", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
plt.legend(loc=3)
plt.ylim([0.8,1.0])

plt.show()

## Gen 1 Vs Gen 2

In [ ]:
frame_no = np.arange(ssim_vs.shape[0])

plt.rcParams["figure.figsize"] = (20,5)

plt.plot(frame_no, ssim_g1_vs, '-s', color='green', label='Generation 1')
plt.plot(frame_no, ssim_g2_vs,'--d', color='blue', label='Generation 2')


plt.title("SSIM: 'Gen 1' Vs 'Gen 2'", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
plt.legend(loc=3)
plt.ylim([0.8,1.0])

plt.show()

## Gen 2 Vs Gen 3

In [ ]:
ssim_g2_vs.shape, ssim_g3_vs.shape

In [ ]:
frame_no = np.arange(ssim_vs.shape[0])

plt.rcParams["figure.figsize"] = (20,5)

plt.plot(frame_no, ssim_g2_vs,'--d', color='blue', label='Generation 2')
plt.plot(frame_no, ssim_g3_vs, ':x', color='m', label='Generation 3')

plt.title("SSIM: 'Gen 1' Vs 'Gen 2'", size=20)
plt.xlabel("Video Sequence No", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
plt.legend(loc=3)
plt.ylim([0.8,1.0])

plt.show()

## Gen 0 Vs Gen 1 Vs Gen 2 Vs Gen 3

In [ ]:
frame_no = np.arange(ssim_vs.shape[0])

plt.rcParams["figure.figsize"] = (20,5)

plt.plot(frame_no, ssim_vs, '-o', color='red', label='Generation 0')
plt.plot(frame_no, ssim_g1_vs, '-s', color='green', label='Generation 1')
plt.plot(frame_no, ssim_g2_vs,'--d', color='blue', label='Generation 2')
plt.plot(frame_no, ssim_g3_vs, ':x', color='m', label='Generation 3')


# plt.title("SSIM: 'Gen 0' Vs 'Gen 1' Vs 'Gen 2' Vs 'Gen 3'", size=20)
plt.xlabel("Video Sequences", size=20)
plt.ylabel("SSIM", size=20)
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
plt.legend()
plt.ylim([0.8,1.0])
plt.tight_layout()

plt.savefig("figures/SSIM_Gen_0_to_3.png")
plt.show()

# SkImage

In [ ]:
from sklearn.model_selection import train_test_split
from skimage import metrics

In [ ]:
GT_sam, _, PD_sam, _ = train_test_split(GT_array, PD_array, test_size=0.92)  ## Randomly sampling 10 video sequence for further anaylisis from Generation 0
GT_sam.shape, PD_sam.shape

In [ ]:
# vs = 0 # Video frame to look for visualtization
for vs in range(GT_sam.shape[0]):
  fig, axes = plt.subplots(2, 17, figsize=(28, 4))
  fig.suptitle(f"Generation 1 Prediction of Video Sequence No: {vs}" , fontsize=20)
  for idx, ax in enumerate(axes[0]):
      ax.imshow(np.squeeze(GT_sam[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
      ax.set_title(f"GT Frame {idx + 4}", size=10)
      ax.axis("off")
  
  for idx, ax in enumerate(axes[1]):
      ax.imshow(np.squeeze(PD_sam[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
      ax.set_title(f"Pred Frame {idx + 4}", size=8)
      ax.axis("off")
  plt.show()

### Struct Simalirity

In [ ]:
GT_sam_scaled = scale_array(GT_sam)
PD_sam_scaled = scale_array(PD_sam)

In [ ]:
str_sim, diff = metrics.structural_similarity(GT_sam_scaled[:,:,:,:,0], PD_sam_scaled[:,:,:,:,0], win_size=3, full=True)
# diff = (diff * 255).astype("uint8")
diff = scale_array(diff)
diff = np.expand_dims(diff, axis=-1)
print(f'Structural Similarity Index: {str_sim}')

In [ ]:
vs = 9
fig, axes = plt.subplots(3, 4, figsize=(10, 8))
fig.suptitle(f"SSIM in Video of Sequence No: {vs}" , fontsize=20)
for idx, ax in enumerate(axes[0]):
    ax.imshow(np.squeeze(GT_sam_scaled[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"GT Frame {idx + 4}", size=14)
    ax.axis("off")

for idx, ax in enumerate(axes[1]):
    ax.imshow(np.squeeze(PD_sam_scaled[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"Pred Frame {idx + 4}", size=14)
    ax.axis("off")

for idx, ax in enumerate(axes[2]):
    ax.imshow(np.squeeze(diff[vs:vs+1, idx:idx+1,:,:,:]), cmap="gray")
    ax.set_title(f"Diff in frame {idx + 4}", size=14)
    ax.axis("off")
plt.tight_layout()

plt.savefig("figures/SSIM_Difference.png")
plt.show()

### Blobs Counting

https://github.com/amirdizche/BlobDetector

In [ ]:
## https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_blob.html#sphx-glr-auto-examples-features-detection-plot-blob-py
# names = ['Laplacian of Gaussian', 'Difference of Gaussian', 'Determinant of Hessian']
from math import sqrt
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray

In [ ]:
### Blobs Counting

LoG_GT, LoG_PD = [], []
DoG_GT, DoG_PD = [], []
ratio_LoG, ratio_DoG = [], []   ## It is the ratio of no ob blobs detected in Predicted image and Blobs detected in Ground Truth

for vs in range(GT_sam_scaled.shape[0]):
  for f in range(GT_sam_scaled.shape[1]):
    img_gt = GT_sam_scaled[vs, f,:,:,0]
    img_pd = PD_sam_scaled[vs, f,:,:,0]
    
    log_gt = blob_log(img_gt, max_sigma=30, num_sigma=10, threshold=.1)
    log_pd = blob_log(img_pd, max_sigma=30, num_sigma=10, threshold=.1)
    dog_gt = blob_dog(img_gt, max_sigma=30, threshold=.1)
    dog_pd = blob_dog(img_pd, max_sigma=30, threshold=.1)

    LoG_GT.append(log_gt.shape[0]) ## Counting no of blobs for LoG in Ground Truth Image
    LoG_PD.append(log_pd.shape[0]) ## Counting no of blobs for LoG in Predicted Image
    DoG_GT.append(dog_gt.shape[0]) ## Counting no of blobs for DoG in Ground Truth Image
    DoG_PD.append(dog_pd.shape[0]) ## Counting no of blobs for DoG in Predicted Image
    ratio_LoG.append(round(log_pd.shape[0]/log_gt.shape[0] ,2))
    ratio_DoG.append(round(dog_pd.shape[0]/dog_gt.shape[0] ,2))

print(f'Average LoG Blob detection ratio in predicted to ground truth image is : {np.average(ratio_LoG)}')
print(f'Average DoG Blob detection ratio in predicted to ground truth image is : {np.average(ratio_DoG)}')

print(f'Standard Deviation of the LoG No of Blob Detected in PD:GT => {np.std(ratio_LoG)}')
print(f'Standard Deviation of the DoG No of Blob Detected in PD:GT => {np.std(ratio_DoG)}')

print(f'Variance in the LoG No of Blob Detected in PD:GT => {np.var(ratio_LoG)}')
print(f'Variance in the DoG No of Blob Detected in PD:GT => {np.var(ratio_DoG)}')

In [ ]:
## laplacian is more noise sensetive
x_ = range(len(ratio_LoG))

mean = np.mean(ratio_LoG)
std = np.std(ratio_LoG)
var = np.var(ratio_LoG)

plt.scatter(x_, ratio_LoG)
plt.plot(x_, ratio_LoG)

plt.axhline(mean, color='r', linestyle='--', label=f'mean = {round(mean,2)}')
plt.axhline(mean+std, color='g', linestyle='--', label='$\pm$'+ f'std = {round(std,4)}')
plt.axhline(mean-std, color='g', linestyle='--')

plt.fill_between(x_, mean-var, mean+var, color='m', alpha=0.5, label=f'variance = {round(var,4)}')

plt.title("Blob Detection Ratio (Laplacian of Gaussian), Standard Deviation, and its Variance", size=20)
plt.xlabel("Image Frames", size=20)
plt.ylabel("Ratio of Blobs Detected: "+ r"$\frac{PD}{GT}$", size=20)
plt.legend(loc=4, fontsize=15)

plt.text(0,0.75, "PD: Predicted Image \nGT: Ground Truth Image", size = 20)

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tight_layout()

plt.savefig('figures/LoG_Blob_detected.png')
plt.show()


In [ ]:
x_ = range(len(ratio_DoG))

mean = np.mean(ratio_DoG)
std = np.std(ratio_DoG)
var = np.var(ratio_DoG)

plt.scatter(x_, ratio_DoG)
plt.plot(x_, ratio_DoG, color='black')

plt.axhline(mean, color='r', linestyle='--', label=f'mean = {round(mean,2)}')
plt.axhline(mean+std, color='r', linestyle='--', label='$\pm$'+ f'std = {round(std,4)}')
plt.axhline(mean-std, color='r', linestyle='--')

plt.fill_between(x_, mean-var, mean+var, color='m', alpha=0.5, label=f'variance = {round(var,4)}' )

plt.title("Blob Detection Ratio (Difference of Gaussian), Standard Deviation, and its Variance", size=20)
plt.xlabel("Image Frames", size=20)
plt.ylabel("Ratio of Blobs Detected: "+ r"$\frac{PD}{GT}$", size=20)
plt.legend(loc=4, fontsize=15)

plt.text(0,0.85, "PD: Predicted Image \nGT: Ground Truth Image", size = 20)

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tight_layout()

plt.savefig('figures/DoG_Blob_detected.png')
plt.show()


In [ ]:
vs = 6  ## Video sequence no
f = 5  ## Frame no

img_gt = GT_sam_scaled[vs, f,:,:,0]
img_pd = PD_sam_scaled[vs, f,:,:,0]

log_gt = blob_log(img_gt, max_sigma=30, num_sigma=10, threshold=.1)
log_pd = blob_log(img_pd, max_sigma=30, num_sigma=10, threshold=.1)
log_gt[:, 2] = log_gt[:, 2] * sqrt(2)   # # Compute radii in the 3rd column.
log_pd[:, 2] = log_pd[:, 2] * sqrt(2)

dog_gt = blob_dog(img_gt, max_sigma=30, threshold=.1)
dog_pd = blob_dog(img_pd, max_sigma=30, threshold=.1)
dog_gt[:, 2] = dog_gt[:, 2] * sqrt(2)   # # Compute radii in the 3rd column.
dog_pd[:, 2] = dog_pd[:, 2] * sqrt(2)

blobs_list = [log_gt, dog_gt, log_pd, dog_pd]
colors = ['m', 'm', 'r', 'r']

titles = [ 'LoG ', 'DoG ', 'LoG ','DoG ',]  # names = ['Laplacian of Gaussian', 'Difference of Gaussian', 'Determinant of Hessian']
# titles = ["Ground Truth", "Predicted", "Ground Truth", "Predicted"]

sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(2, 2, figsize=(9, 9), sharex=True, sharey=True)

ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
  # ax[idx].set_title(title+f"\nFrame: {f} of Video {vs}", size = 20)
  ax[idx].set_xticks([])
  ax[idx].set_yticks([])

  if idx < 2:
    ax[idx].imshow(img_gt, 'gray')
    ax[0].set_ylabel('Ground Truth Frames', size=20)
    ax[idx].set_title("Blobs Counting using: "+title+f"\nFrame: {f} of Video {vs}", size = 20)

  else:
    ax[idx].imshow(img_pd, 'gray')
    ax[2].set_ylabel('Predicted Frames', size=20)

  for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, linestyle='--', fill=False)
        ax[idx].add_patch(c)
  # ax[idx].set_axis_off()
plt.tight_layout()

plt.savefig("figures/Blobs_counting.png")
plt.show()

### Shape Curvature

In [ ]:
from scipy import ndimage as ndi
from skimage.feature import shape_index
from skimage.draw import disk

[1] https://www.sciencedirect.com/science/article/abs/pii/026288569290076F?via%3Dihub

[2] https://journals.aps.org/prb/abstract/10.1103/PhysRevB.78.024113

The shape index is a single valued measure of local curvature of image microstructure and it is mathematically obtained from the eigen values of the Hessian [1], defined by Koenderink & van Doorn

In [ ]:
## Shape Index
## https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_shape_index.html#sphx-glr-auto-examples-features-detection-plot-shape-index-py

vs = 2
f = 11
pd_img = PD_sam[vs,f,:,:,0]
gt_img = GT_sam[vs,f,:,:,0]

#############################################################################
s = shape_index(gt_img)

# In this example we want to detect 'spherical caps',
# so we threshold the shape index map to
# find points which are 'spherical caps' (~1)

target = 1
delta = 0.05

point_y, point_x = np.where(np.abs(s - target) < delta)
point_z = gt_img[point_y, point_x]

# The shape index map relentlessly produces the shape, even that of noise.
# In order to reduce the impact of noise, we apply a Gaussian filter to it,
# and show the results once in

s_smooth = ndi.gaussian_filter(s, sigma=0.5)

point_y_s, point_x_s = np.where(np.abs(s_smooth - target) < delta)
point_z_s = gt_img[point_y_s, point_x_s]


fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(1, 3, 1)

ax1.imshow(gt_img, cmap=plt.cm.gray)
ax1.axis('off')
ax1.set_title('Ground Truth Image')
scatter_settings = dict(alpha=0.75, s=10, linewidths=0)
ax1.scatter(point_x, point_y, color='blue', **scatter_settings)
ax1.scatter(point_x_s, point_y_s, color='green', **scatter_settings)

ax2 = fig.add_subplot(1, 3, 2, projection='3d', sharex=ax1, sharey=ax1)
x, y = np.meshgrid(np.arange(0, gt_img.shape[0], 1), np.arange(0, gt_img.shape[1], 1))
ax2.plot_surface(x, y, gt_img, linewidth=0, alpha=0.5, color="green")
ax2.scatter(point_x, point_y, point_z, color='blue', label='$|s - 1|<0.05$', **scatter_settings)
ax2.scatter(point_x_s, point_y_s, point_z_s, color='green', label='$|s\' - 1|<0.05$', **scatter_settings)
ax2.legend(loc='lower left')
ax2.axis('off')
ax2.set_title('3D visualization')

ax3 = fig.add_subplot(1, 3, 3, sharex=ax1, sharey=ax1)
ax3.imshow(s, cmap=plt.cm.gray)
ax3.axis('off')
ax3.set_title(r'Shape index, $\sigma=1$')

fig.tight_layout()
plt.savefig('figures/shape_ind_GT.png')
plt.show()

#############################################################

s = shape_index(pd_img)

target = 1
delta = 0.05

point_y, point_x = np.where(np.abs(s - target) < delta)
point_z = pd_img[point_y, point_x]

# The shape index map relentlessly produces the shape, even that of noise.
# In order to reduce the impact of noise, we apply a Gaussian filter to it,
# and show the results once in

s_smooth = ndi.gaussian_filter(s, sigma=0.5)

point_y_s, point_x_s = np.where(np.abs(s_smooth - target) < delta)
point_z_s = pd_img[point_y_s, point_x_s]


fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(1, 3, 1)

ax1.imshow(pd_img, cmap=plt.cm.gray)
ax1.axis('off')
ax1.set_title('Predicted Image')
scatter_settings = dict(alpha=0.75, s=10, linewidths=0)
ax1.scatter(point_x, point_y, color='blue', **scatter_settings)
ax1.scatter(point_x_s, point_y_s, color='green', **scatter_settings)

ax2 = fig.add_subplot(1, 3, 2, projection='3d', sharex=ax1, sharey=ax1)
x, y = np.meshgrid(np.arange(0, pd_img.shape[0], 1), np.arange(0, pd_img.shape[1], 1))
ax2.plot_surface(x, y, pd_img, linewidth=0, alpha=0.5, color="red")
ax2.scatter(point_x, point_y, point_z, color='blue', label='$|s - 1|<0.05$', **scatter_settings)
ax2.scatter(point_x_s, point_y_s, point_z_s, color='green', label='$|s\' - 1|<0.05$', **scatter_settings)
ax2.legend(loc='lower left')
ax2.axis('off')
ax2.set_title('3D visualization')

ax3 = fig.add_subplot(1, 3, 3, sharex=ax1, sharey=ax1)
ax3.imshow(s, cmap=plt.cm.gray)
ax3.axis('off')
ax3.set_title(r'Shape index, $\sigma=1$')

fig.tight_layout()
plt.savefig('figures/shape_ind_PD.png')
plt.show()

###  Edge-based and region-based segmentation

In [ ]:
vs = 2
f = 11
pd_img = scale_array(PD_sam[vs,f,:,:,0])
gt_img = scale_array(GT_sam[vs,f,:,:,0])

from skimage.exposure import histogram

hist1, hist_centers1 = histogram(gt_img)
hist2, hist_centers2 = histogram(pd_img)

fig = plt.figure(figsize=(8, 8)) # adjust figure size as needed
# create subplots

ax1 = fig.add_subplot(2, 1, 1) # column 1 and 2, row 1
ax2 = fig.add_subplot(2, 2, 3) # column 1, row 2
ax3 = fig.add_subplot(2, 2, 4) # column 2, row 2

ax1.plot(hist_centers1, hist1, lw=2, color='red', label='GT')
ax1.plot(hist_centers2, hist2, lw=2, color="blue", label='PD')
ax1.set_title('histogram of gray values', size = 20)
ax1.set_ylabel("No. of Pixels", size= 20)
ax1.set_xlabel("Pixel's Intensity Value (0 to 255)", size= 20)
ax1.legend(loc=0, fontsize=12)

ax2.imshow(gt_img, cmap=plt.cm.gray)
ax2.axis('off')
ax2.set_title(f"Ground Truth Frame {f} of VS {vs}  " , size=12)

ax3.imshow(pd_img, cmap=plt.cm.gray)
ax3.axis('off')
ax3.set_title(f"Predicted Image Frame {f} of VS {vs} ", size=12)

ax1.tick_params(axis='both', which='major', labelsize=18)

plt.subplots_adjust(hspace=0.5)
# display the plot

# ax2.text(-20,73, " The x-axis represents the pixel intensity values, while the y-axis represents the number of pixels that have that intensity value." )
plt.savefig('figures/histogram.png')
plt.show()


In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(8, 3), sharey=True)

# axes[0].imshow(img1>150 , cmap=plt.cm.gray)
# axes[0].set_title('Original  > 150')

# axes[1].imshow(img2 > 150, cmap=plt.cm.gray)
# axes[1].set_title('Predicted > 150')

# for a in axes:
#     a.axis('off')

# plt.tight_layout()